In [2]:
from Player import Player
from Trainer import Trainer
from TetrisModel import TetrisModel
from Pretrainer import Pretrainer
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import time

In [3]:
piece_dim = 8
key_dim = 12
depth = 16
gamma = 0.99
lam = 0.95

In [ ]:
# Use lambda instead of gamma to immitate shape of gae without value predictions
pretrainer = Pretrainer(gamma=lam)

In [ ]:
players_data = pretrainer._load_data()

In [5]:
pretrainer._load_dset(players_data)

1.00

In [4]:
# max_len = pretrainer._max_len
max_len = 7

In [ ]:
gt_dset = pretrainer._cache_dset()

In [5]:
agent = TetrisModel(piece_dim=piece_dim,
                    key_dim=key_dim,
                    depth=depth,
                    num_heads=4,
                    num_layers=4,
                    max_length=max_len)

In [6]:
logits, values = agent((tf.random.uniform((1, 28, 10, 1)),
                        tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                        tf.random.uniform((1, max_len-1), minval=0, maxval=key_dim, dtype=tf.int32)))
agent.summary(), tf.shape(logits), tf.shape(values)

Model: "tetris_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (1, 70, 16)               4800      
                                                                 
 seq_embedding (SeqEmbedding  multiple                 128       
 )                                                               
                                                                 
 seq_embedding_1 (SeqEmbeddi  multiple                 192       
 ng)                                                             
                                                                 
 pdec_0 (DecoderLayer)       multiple                  9776      
                                                                 
 pdec_1 (DecoderLayer)       multiple                  9776      
                                                                 
 pdec_2 (DecoderLayer)       multiple                 

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1,  6, 12])>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 6, 1])>)

In [7]:
# agent_optimizer = keras.optimizers.Adam(learning_rate=1e-5)
# agent.compile(optimizer=agent_optimizer)

In [8]:
ref_agent = TetrisModel(piece_dim=piece_dim,
                        key_dim=key_dim,
                        depth=depth,
                        num_heads=4,
                        num_layers=4,
                        max_length=max_len)

In [9]:
logits, values = ref_agent((tf.random.uniform((1, 28, 10, 1)),
                            tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                            tf.random.uniform((1, max_len-1), minval=0, maxval=key_dim, dtype=tf.int32)))

In [10]:
agent.load_weights('agent_weights_small/agent')

In [11]:
ref_agent.load_weights('agent_weights_small/agent')

In [11]:
# epochs = 10

In [14]:
# losses, accs = pretrainer.train(agent, gt_dset, epochs)

In [14]:
# plt.plot(losses)
# plt.plot(accs)

In [25]:
if 'y' in input('YOU SURE?????'):
    agent.save_weights('agent_weights_small/agent_finetuned')

YOU SURE????? y


In [12]:
if 'y' in input('YOU SURE?????'):
    ref_agent.save_weights('agent_weights_small/agent_reference')

YOU SURE????? y


In [12]:
%matplotlib qt

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=1e-5)

In [14]:
trainer = Trainer(model=agent,
                  ref_model=ref_agent,
                  optimizer=optimizer,
                  max_len=max_len,
                  gamma=gamma,
                  lam=lam,
                  max_episode_steps=100,
                  buffer_cap=500)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michaelsherrick. Use `wandb login --relogin` to force relogin


In [15]:
trainer.fill_replay_buffer()

Done filling replay buffer

In [ ]:
trainer.train(gens=10000, train_steps=5)

Current Gen: 1	|	Avg Reward: 0.0	|	Total Reward: 0.4	|WARNING:tensorflow:From C:\Users\micha\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\autograph\impl\api.py:377: ReplayBuffer.get_next (from tf_agents.replay_buffers.replay_buffer) is deprecated and will be removed in a future version.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
PPO Loss: 0.05	|	KL Penalty: 0.06	|	Critic Loss: 0.36	|	|

In [ ]:
episode_data = trainer.player.run_episode(agent, max_steps=100, greedy=True, renderer=trainer.renderer)

In [18]:
episode_boards, episode_pieces, episode_inputs, episode_actions, episode_probs, episode_valid, episode_values, episode_rewards = episode_data

In [19]:
episode_advantages, episode_returns = trainer._compute_gae(episode_values, episode_rewards, trainer.gamma, trainer.lam)

In [20]:
fig, ax = plt.subplots()
ax.plot(episode_returns, label='Returns')
ax.plot(episode_rewards, label='Rewards')
ax.plot(episode_values, label='Values')
ax.plot(episode_advantages, label='Advantages')
ax.legend()
tf.reduce_sum(episode_rewards)

<tf.Tensor: shape=(), dtype=float32, numpy=6.5>